In [34]:
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.inspection import permutation_importance
from matplotlib import pyplot as plt
from datetime import datetime

In [2]:
df = pd.read_csv('bankingchurn_data_modif.csv', error_bad_lines=False, sep=';')

C:\Users\Goodie\AppData\Local\Temp\ipykernel_17952\3333554029.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('bankingchurn_data_modif.csv', error_bad_lines=False, sep=';')
C:\Users\Goodie\AppData\Local\Temp\ipykernel_17952\3333554029.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('bankingchurn_data_modif.csv', error_bad_lines=False, sep=';')


In [3]:
df

,contract_start,contract_end,date_of_birth,gender,profession,size_household,ZIP,segment,credit_rating,no_credit_rating_flag,...,online_number_of_logins_per_month,online_tranactions_per_month,advisor_contacts_last12months,customer_limit,cash_withdraws_per_month,cash_withdrawals_value,consumer_credit_value,consumer_credit_maturity,account_fee,number_of_refusals
0,2010-10-24,NaN,1991-02-11,M,Professional Services,1,76829,S2,"89,5685176247469",0,...,0,0,3,1000,4,"459,840939712736","-24011,393551989",205,15,6
1,2010-04-30,2017-10-20,1986-04-06,W,NaN,1,35104,S2,"94,2705069107966",0,...,16,8,2,1000,7,"476,334458774319","-5368,74306807524",137,15,6
2,2013-05-02,NaN,1993-04-18,M,Transportation,1,63450,S2,"93,0309465957283",0,...,26,12,2,1000,3,"332,640202190496","-9678,71341935483",624,15,4
3,2016-12-24,NaN,1994-07-29,W,Professional Services,2,9573,S3,NaN,1,...,0,0,4,1050,4,"378,32474448561","-12110,3059156975",1263,10,7
4,2011-10-01,NaN,1990-10-28,W,Research,1,66916,S2,"90,45766439571",0,...,23,18,3,1000,4,"471,509299299279",0,521,10,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,2006-09-11,NaN,1985-03-02,M,Finance,2,59505,S1,NaN,1,...,0,0,4,1550,7,"20,197055391671",0,1373,0,0
99996,2005-04-04,NaN,1979-09-28,W,Construction,1,51143,S3,"96,5343938026365",0,...,0,0,1,1000,4,"294,554004818315","-22473,9139129287",810,10,3
99997,2017-09-05,NaN,1996-04-25,W,Construction,1,6577,S2,"85,982146943176",0,...,3,0,1,1000,3,"109,991597571823",0,1063,15,2
99998,2008-10-31,NaN,1988-03-23,W,Information Technology,1,79115,S1,NaN,1,...,31,23,3,1750,3,"299,817586666007",0,0,0,4


In [4]:
df.isna().sum()

contract_start                           0
contract_end                         95129
date_of_birth                            0
gender                                   0
profession                            7729
size_household                           0
ZIP                                      0
segment                                  0
credit_rating                        22483
no_credit_rating_flag                    0
main_account_flag                        0
online_banking_flag                      0
tele_banking_flag                        0
creditcard_flag                          0
insurance_life_flag                      0
insurance_house_flag                     0
insurance_car_flag                       0
insurance_other_flag                     0
mortgage_flag                            0
portfolio_flag                           0
last_balance                             0
last_balance_minus_6_months              0
last_balance_minus_12_months             0
income_sala

In [6]:
profile = ProfileReport(df, title="Pandas Profiling Report")
profile.to_file("your_report.html")

Summarize dataset:   0%|          | 0/58 [00:00<?, ?it/s]

C:\Users\Goodie\anaconda3\lib\site-packages\scipy\stats\_stats_py.py:5215: RuntimeWarning: overflow encountered in longlong_scalars
  (2 * xtie * ytie) / m + x0 * y0 / (9 * m * (size - 2)))


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [37]:
pd.to_numeric(df['contract_end'],"%Y-%m-%d")

ValueError: invalid error value specified

In [36]:
df['churner'] = np.where(df['contract_end'].isna(), 0, 1)
df['duration_contract'] = np.where(df['contract_end'].isna(),
                              datetime.now()-pd.to_numeric(df['contract_end'],"%Y-%m-%d"),
                            pd.to_numeric(df.contract_end,"%Y-%m-%d")-pd.to_numeric(df.contract_start,"%Y-%m-%d"))

ValueError: invalid error value specified

In [26]:
df.sample(10)

,contract_start,contract_end,date_of_birth,gender,profession,size_household,ZIP,segment,credit_rating,no_credit_rating_flag,...,online_tranactions_per_month,advisor_contacts_last12months,customer_limit,cash_withdraws_per_month,cash_withdrawals_value,consumer_credit_value,consumer_credit_maturity,account_fee,number_of_refusals,churner
51231,2003-06-09,NaN,1977-07-29,M,NaN,2,06917,S2,"98,2035311734915",0,...,0,2,"1071,22052966063",4,"341,038183940611",0,1007,15,6,0
58854,2006-01-10,NaN,1981-01-29,W,Real Estate,1,61231,S2,NaN,1,...,15,0,1000,1,"280,941528636642","-8747,25220265698",1218,10,1,0
66385,2009-07-22,NaN,1983-12-17,W,Education,2,53534,S2,NaN,1,...,4,0,1500,4,"28,5606600205078",0,0,10,1,0
37473,2010-08-24,2017-10-25,1984-10-01,W,Retiree,1,92540,S5,"87,4123625653405",0,...,23,5,"1038,88070290603",3,"524,601065995608",0,1011,10,12,1
28115,2013-03-20,NaN,1985-01-15,W,Nonprofit,1,99986,S1,"96,9023408548894",0,...,18,1,1000,5,"322,917401960356","-11839,0011149618",1129,10,11,0
88221,2011-06-18,NaN,1979-08-15,W,NaN,1,86356,S2,"94,0055834919773",0,...,23,3,1350,4,"519,288339082592",0,0,15,11,0
83663,2007-01-26,NaN,1990-03-24,W,Student,1,54516,S4,"93,9032178686955",0,...,12,3,1350,3,"296,393398126294",0,0,10,7,0
21897,2002-03-10,NaN,1984-01-19,W,Telecommunications,1,37318,S1,NaN,1,...,0,0,1000,6,"380,766823757",0,801,10,9,0
72925,2011-08-29,NaN,1975-10-15,W,NaN,1,66113,S2,NaN,1,...,10,1,1000,4,"473,538248668357",0,0,10,5,0
41243,2011-08-20,2018-05-16,1991-10-17,W,Finance,1,25491,S2,"99,0106116289662",0,...,0,0,"1118,26107949462",2,0,"-3112,03397168954",1007,10,5,1


In [30]:
# pd.to_datetime(df.date_of_birth,'%Y-%m-%d')
df.columns

Index(['contract_start', 'contract_end', 'date_of_birth', 'gender',
       'profession', 'size_household', 'ZIP', 'segment', 'credit_rating',
       'no_credit_rating_flag', 'main_account_flag', 'online_banking_flag',
       'tele_banking_flag', 'creditcard_flag', 'insurance_life_flag',
       'insurance_house_flag', 'insurance_car_flag', 'insurance_other_flag',
       'mortgage_flag', 'portfolio_flag', 'last_balance',
       'last_balance_minus_6_months', 'last_balance_minus_12_months',
       'income_salary_per_year', 'income_deposits_per_year',
       'income_securities_per_year', 'insurance_life_premium_per_month',
       'insurance_house_premium_per_year', 'insurance_car_premium_per_year',
       'mortgage_value', 'mortgage_interest', 'mortgage_downpayment',
       'mortgage_dayuntilmaturity', 'mortgage_numbers',
       'online_number_of_logins_per_month', 'online_tranactions_per_month',
       'advisor_contacts_last12months', 'customer_limit',
       'cash_withdraws_per_month', '

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 45 columns):
 #   Column                             Non-Null Count   Dtype 
---  ------                             --------------   ----- 
 0   contract_start                     100000 non-null  object
 1   contract_end                       4871 non-null    object
 2   date_of_birth                      100000 non-null  object
 3   gender                             100000 non-null  object
 4   profession                         92271 non-null   object
 5   size_household                     100000 non-null  int64 
 6   ZIP                                100000 non-null  object
 7   segment                            100000 non-null  object
 8   credit_rating                      77517 non-null   object
 9   no_credit_rating_flag              100000 non-null  int64 
 10  main_account_flag                  100000 non-null  int64 
 11  online_banking_flag                100000 non-null  i